# Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных, таблицы:
##### Таблица books

##### Таблица authors

##### Таблица publishers

##### Таблица ratings

##### Таблица reviews


## Задания
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


### Загрузка таблиц

In [2]:
query = '''
SELECT * FROM books LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

In [3]:
query = '''
SELECT * FROM authors LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

In [4]:
query = '''
SELECT * FROM publishers LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства;

In [5]:
query = '''
SELECT * FROM ratings LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

In [6]:
query = '''
SELECT * FROM reviews LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

### Задания

#### Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
SELECT 
    COUNT(*)
FROM
    books
WHERE 
    publication_date > '2000-01-01';
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

#### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
query = '''

SELECT b.book_id, 
       b.title, 
       COUNT(DISTINCT rv.review_id) AS cnt_reviews, 
       ROUND(AVG(r.rating),3) AS avg_rating
FROM books AS b

LEFT JOIN reviews AS rv
ON b.book_id = rv.book_id

LEFT JOIN  ratings AS r
ON b.book_id = r.book_id

GROUP BY b.book_id, b.title
ORDER BY cnt_reviews DESC

'''

pd.io.sql.read_sql(query, con = engine)


,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.663
1,963,Water for Elephants,6,3.977
2,734,The Glass Castle,6,4.207
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.415
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667
996,808,The Natural Way to Draw,0,3.000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000
998,221,Essential Tales and Poems,0,4.000


In [9]:
query = '''
SELECT
    books.title AS title, -- выведем название книги
    AVG(ratings.rating) AS avg_rating, --средний рейтинг
    COUNT(reviews.review_id) AS cnt_review_id -- количество обзоров
FROM -- к таблице books добавим табилцы ratings и reviews по полю book_id
    books
LEFT OUTER JOIN ratings ON ratings.book_id = books.book_id
LEFT OUTER JOIN reviews ON reviews.book_id = books.book_id
GROUP BY 
    books.title
ORDER BY -- сортировка по количеству обзоров
    cnt_review_id DESC;
'''
pd.io.sql.read_sql(query, con = engine)

,title,avg_rating,cnt_review_id
0,Twilight (Twilight #1),3.662500,1120
1,The Hobbit or There and Back Again,4.125000,528
2,The Catcher in the Rye,3.825581,516
3,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,492
4,Harry Potter and the Chamber of Secrets (Harry...,4.287500,480
...,...,...,...
994,The Natural Way to Draw,3.000000,0
995,Essential Tales and Poems,4.000000,0
996,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,0
997,Leonardo's Notebooks,4.000000,0


Лидер по обзорам Twilight - 1120 обзоров. Средний рейтинг 3,66.

#### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [10]:
query = '''
SELECT 
    publishers.publisher AS publisher, -- выведем название издательства 
    COUNT(books.book_id) AS cnt_books --количество книг
FROM -- к таблице books добавим табилцу publishers по полю publisher_id
    books
LEFT OUTER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE -- страниц в книге >50
    num_pages > '50'
GROUP BY
    publisher
ORDER BY -- сортировка по количеству книг
    cnt_books DESC
LIMIT 1; -- первая строка=лидер
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,cnt_books
0,Penguin Books,42


Издательство, которое издало наибольшее число книг толще 50 страниц - Penguin Books - 42 книги

#### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [11]:
query = '''
SELECT 
    aut_avg.author,
    AVG(aut_avg.avg_rating) AS avg_avg_rating
FROM
        (SELECT 
            authors.author AS author
           , AVG(ratings.rating) AS avg_rating
        FROM books
            LEFT JOIN authors ON books.author_id = authors.author_id
            LEFT JOIN ratings ON books.book_id = ratings.book_id
        GROUP BY 
             books.book_id
             , authors.author_id
        HAVING
            COUNT(ratings.rating_id) >=50 
        ) AS aut_avg 
GROUP BY    
    aut_avg.author
ORDER BY
    avg_avg_rating DESC
LIMIT 1; -- первая строка=лидер
                    

'''
pd.io.sql.read_sql(query, con = engine)

,author,avg_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré - 4,28!

#### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [12]:
query = '''
SELECT -- среднее по количеству обзоров
    AVG(Sub.cnt_reviews) AS avg_reviews
FROM
    (SELECT  -- выбираем username 
        reviews.username,
        COUNT(reviews.review_id) AS cnt_reviews
    FROM -- из таблицы с обзорами
        reviews
    WHERE -- по username
        username IN
            (SELECT -- выбираем username
                ratings.username
            FROM -- из таблицы с оценками
                ratings
            GROUP BY
                ratings.username
            HAVING --  поставили более 50 оценок
                COUNT(ratings.rating_id) > '50')
    GROUP BY
        reviews.username) AS Sub;
'''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.333333


Среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок - 24.33.

## Вывод

- Всего в каталоге 1000 книг.
- С 1 января 2000 года вышла 819 книг. 
- Книги с максимальным количеством обзоров:
    - "Сумерки" (7 обзоров и средний рейтинг 3,66), 
    - "Ешь, молись, люби" (6 обзоров и средний рейтинг 3,39), 
    - 2-я и 3-я книги о Гарри Поттере (6 обзоров на каждую и средний рейтинг 4,29 и 4,41, соответственно) 
    - "Чужестранка" (6 обзоров и оценка 4,13)
    - есть книги без обзоров - 6
- Издательство, которое издало наибольшее число книг (толще 50 страниц) - Penguin Books - 42 книги
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré - 4,28
- Среднее количество текстовых обзоров активных пользователей (поставили >50 оценок) - на 24.33 книги.
